# Strategy Report Notebook
This notebok runs analysis on a given strategy.

## Import Libraries

In [ ]:
import pandas as pd
import ipywidgets as widgets
import datapane as dp 
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
from EquityHedging.reporting.excel import sheets
import os

## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for Equity Benchmark

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**):  SPTR, M1WD, SX5T

In [ ]:
equity_bmk = 'M1WD'

#create returns data dictionary for equity benchmark
bmks_dict= dm.get_equity_hedge_returns(equity_bmk, only_equity=True)

## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for a Strategy
To import the returns data:
* In **new_data = get_new_strategy_returns_data( report_name= , sheet_name = ,index_col=0)**
    * Add File Name to **report_name** 
    * Add sheet name to **sheet_name**

In [ ]:
new_data= dm.get_new_strategy_returns_data( report_name = "Credit Suisse_SX5E_Tail_FSFVA_20210528_UPS Pension.xlsx", 
                                           sheet_name = "data",index_col=0)

#create returns data dictionary for strategy
new_data_dict = dm.get_data_dict(new_data,'index')

#merge dictionaries
ret_data_dict = dm.merge_dicts(bmks_dict,new_data_dict)

## View Returns Data
Select frequency (Daily, Weekly, Monthly, Yearly) to view data

In [ ]:
freq_list = ['Daily', 'Weekly', 'Monthly','Quarterly', 'Yearly']
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(ret_data_dict[frequency])

## Compute Return Stats and hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(ret_data_dict)

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(ret_data_dict[frequency])

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(ret_data_dict)

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

# Compute Correlations

In [ ]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict = summary.get_corr_data(ret_data_dict)

# Display Correlations

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['corr', 'corr_down', 'corr_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(ret_data_dict)

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quantile Analysis

In [ ]:
group_list=['Quintile','Decile']
@interact
def display_returns(group = group_list):
    quintile_df = summary.get_grouped_data(ret_data_dict,[], False, group)
    return(quintile_df.style.format("{:.2%}"))

## Display Quantile Analysis
Display Quintile Analysis Bar Chart:
* Compare strategies to the Equity Benchmark's returns

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list, strat=list(ret_data_dict['Monthly'].select_dtypes('float').columns)[1:]):    
    df = summary.get_grouped_data(ret_data_dict, [], False, group)
    
    for col in df.columns:
        if not (col == equity_bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()
    

# Create Report

In [ ]:
strat_report = 'Strat_Report'
selloffs = True
rp.generate_strat_report(strat_report, ret_data_dict, selloffs)